# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("data.csv")

# Задача: выяснить, влияет ли семейное положение клиента и количество детей на факт погашения кредита в срок

Необходимо ответить на следующие вопросы:
* Есть ли зависимость между количеством детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

Имеется датасет из банка, согласно описанию он имеет следующие столбцы:
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

Рассмотрим первые 5 строк таблиицы:

In [3]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Из рассмотрения первых пяти строк видно, что в столбце `days_employed` присутствуют артефакты, поскольку это значение очевидно не может быть отрицательным. Также в столбце `education` присутствуют неявные дубликаты "среднее" и "Среднее", что также необходимо будет исправить при дальнейшей обработке данных.

Можно заметить, что пары столбцов `education` - `education_id` и `family_status` - `family_status_id` содержат одну и ту же информацию. Для удобства можно будет убрать один стобец из каждой пары, создав словарь id: значение.

Помимо этого данные в столбцах `days_employed` и `total_income`представлены в неудобном дробном формате. Необходимо будет перевести их в целочисленный тип.


Рассмотрим общую информацию о таблтце:

In [4]:
data["dob_years"].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Можно заметить, что в стобцах `days_employed` и `total_income` присутствуют пропущенные значения. В стобце `total_income`Они могут быть вызваны тем фактом, что человек не имеет стабильного ежемесячного дохода или не работал официально. В `days_employed` они могут быть следствием того, что человек не работал официально или вообще никогда не работал. Пропуски в этих столбцах можно заполнить медианным значением. Замена на меианное значение в данном случае будет уместно, поскольку это значение не будет уводить объект в сторну с лучшей кредитной историей или худшей.

Узнаем, есть ли аномальные значения в столбце `total_income`:

In [6]:
data.loc[data["total_income"] <= 0, "total_income"].count()

0

Таких занчений нет.

Исследуем на артефакты значения столбца `dob_years`:

In [7]:
data["dob_years"].sort_values().unique()

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75])

Значение 0 очевидно является ошибочным. Посмотрим, какую часть данных составляют подобные артефакты:

In [8]:
data.loc[data["dob_years"] == 0, "dob_years"].count() / data["dob_years"].count()

0.004692218350754936

Таких значений крайне мало, они будут заменены на медианное значение по датасету в дальнейшем.

Исследуем на артефакты столбец `children`:

In [9]:
data["children"].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Значения -1 и 20 очевидно являются артефактами. -1 может означать отсутствие информации о количестве детей. значение 20, скорее всего, появилось из-за ошибки заполнения.

Проверим какую часть данных составляют эти артефакты:

In [10]:
data.loc[(data["children"] == -1) | (data["children"] == 20), "children"].count() / data["children"].count()

0.005714285714285714

Данные составляют незначительную часть датасета и будут заменены на медианное значение.

Проверим данные на наличие дублирующихся строк:

In [11]:
data.duplicated().sum()

54

В датасете присутствуют дубликаты. Несмотря на то, что в данной задаче возможно появление дубликатов, будем считать, что они вызваны ошибками при заполнении таблицы, потому что шанс полного совпадения информации о двух разных людях крайне мал. Эти строки будут удалены при дальнейшей обработке. Удаление такого маленького количества строк (в сравнении с количеством строк самого датасета) не должно сильно повлиять на результат исследования.

Проверим таблицу на наличие неявных дубликатов во всех столбцах, в которых они могут возникнуть:

In [12]:
data["education"].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [13]:
data["family_status"].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [14]:
data["gender"].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [15]:
data["income_type"].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [16]:
data["purpose"].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Неявные дубликаты присутствуют в столбцах `education` и `purpose`. Значения в стобце `purpose` следует категориализовать в дальнейшем.

Посмотрим какую часть данных составляют пропущенные и отрицательные значения в столбце `days_employed`:

In [17]:
print(data.loc[(data["days_employed"] < 0 | data["days_employed"].isna()), "days_employed"].count()/data.shape[0])

0.7389547038327526


Анамалии и пропуски занимают большую часть данных, поэтому, если просто удалить их, то потеряется слишклм много информации. Скорее всего, отрицательные значения - ошибка заполнения таблице, на их месте должны были быть положительные. Поэтому при обработке таблицы заменим отрицательные занченя на положительные, а пропуски заменим на медианное значение.

### Шаг 2.1 Исправление аномалий в данных

Исправим отрицательные значения в столбце `days_employed` на положительные:

In [18]:
data.loc[data["days_employed"] < 0, "days_employed"] = -1 * data.loc[data["days_employed"] < 0, "days_employed"]

Проверим результат исправления:

In [19]:
data.loc[data["days_employed"] < 0, "days_employed"].count()

0

Заменим аномальные значения столбца `dob_years` на медианное значения:

In [20]:
data.loc[data["dob_years"] == 0, "dob_years"] = data.loc[data["dob_years"] != 0, "dob_years"].median()

Проверим результат замены:

In [21]:
data["dob_years"].sort_values().unique()

array([19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
       70, 71, 72, 73, 74, 75])

Исправим аномальные значения в столбце `children` на медианное значение:

In [22]:
data.loc[(data["children"] == -1) | (data["children"] == 20), "children"] = data.loc[(data["children"] != -1) | (data["children"] != 20), "children"].median()

Проверим результат замены:

In [23]:
data["children"].unique()

array([1, 0, 3, 2, 4, 5])

### Шаг 2.2 Заполнение пропусков 

Заполним пропуски в столбце `days_employed` медианным значением:

In [24]:
data["days_employed"] = data["days_employed"].fillna(data["days_employed"].median())

Проверим результат заполнения:

In [25]:
data["days_employed"].isna().sum()

0

Заполним пропуски в столбце `total_income` медианным значением:

In [26]:
data["total_income"] = data["total_income"].fillna(data["total_income"].median())

Проверим результат заполнения:

In [27]:
data["total_income"].isna().sum()

0

### Шаг 2.3. Изменение типов данных.

Поменяем тип данных столбцов `total_income` и `days_employed` на целочисленный:

In [28]:
data["total_income"] = data["total_income"].astype("int64")
data["days_employed"] = data["days_employed"].astype("int64")

Проверим, выполнилась ли замена:

In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

Удалим дублирующиеся строки и восстановим индексы:

In [30]:
data = data.drop_duplicates()
data = data.reset_index(drop = True)

Проверим, удалились ли дубликаты:

In [31]:
data.duplicated().sum()

0

Приведём значения столбца `education` к одному виду:

In [32]:
data["education"] = data["education"].str.lower()

Проверим результат:

In [33]:
data["education"].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Оставим в датасете только идентефикаторы `education_id` и `family_status_id` столбцов `education` и `family_status` соответственно. Расшифровки этих айди вынесем в отдельныt датафреймs словари:

In [34]:
data_dict_education = data[["education_id", "education"]].drop_duplicates().reset_index(drop=True)
data_dict_family_status = data[["family_status_id", "family_status"]].drop_duplicates().reset_index(drop=True)
data = data[["children", "days_employed", "dob_years", "education_id", "family_status_id", "gender", "income_type", "debt", "total_income", "purpose"]]

In [35]:
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


In [36]:
data_dict_education.head()

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [37]:
data_dict_family_status.head()

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


### Шаг 2.6. Категоризация дохода.

Создадим столбец `total_income_category` со следующими категориями:
* 0–30000 — 'E'
* 30001–50000 — 'D'
* 50001–200000 — 'C'
* 200001–1000000 — 'B'
* 1000001 и выше — 'A'

In [38]:
def make_income_category(value):
    if value <= 30000:
        return "E"
    elif value <= 50000:
        return "D"
    elif value <= 200000:
        return "C"
    elif value <= 1000000:
        return "B"
    else:
        return "A"
    
data["total_income_category"] = data["total_income"].apply(make_income_category)

Проверка:

In [39]:
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.7. Категоризация целей кредита.

Создадим столбец `purpose_category` со следующими категориями:
* операции с автомобилем
* операции с недвижимостью
* проведение свадьбы
* получение образования

In [40]:
def make_purpose_category(string):
    if "авто" in string:
        return "операции с автомобилем"
    elif "недвиж" in string or "жиль" in string:
        return "операции с недвижимостью"
    elif "сва" in string:
        return "проведение свадьбы"
    elif "образ" in string:
        return "получение образования"
    else:
        return "другое"
    
data["purpose_category"] = data["purpose"].apply(make_purpose_category)

Проверим таблицу и посмотрим в скольких строках записано 'другое' (сколько строк не удалось корректно заполнить):

In [41]:
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


In [42]:
data.loc[data["purpose_category"] == "другое", "purpose"].count()

0

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для того, чтобы проверить это рассмотрим процент людей, вернувших кредит в срок для трех разных категорий: нет детей, 1-2 ребенка, больше 2 детей.

In [43]:
first_category_percent = data.loc[(data["children"] == 0) & (data["debt"] == 0), "debt"].count() / data.loc[(data["children"] == 0), "debt"].count()
second_category_percent = data.loc[(data["children"] > 0) & (data["children"] < 3) & (data["debt"] == 0), "debt"].count() / data.loc[(data["children"] > 0) & (data["children"] < 3), "debt"].count()
third_category_percent = data.loc[(data["children"] > 2) & (data["debt"] == 0), "debt"].count() / data.loc[(data["children"] > 2), "debt"].count()
first_category_size = data.loc[(data["children"] == 0), "debt"].count()
second_category_size = data.loc[(data["children"] > 0) & (data["children"] < 3), "debt"].count()
third_category_size = data.loc[(data["children"] > 2), "debt"].count()

Построим таблицу для более удобного восприятия информации:

In [44]:
table = [[first_category_percent, first_category_size], [second_category_percent, second_category_size], [third_category_percent, third_category_size]]
index = ["Нет детей" , "Один - два ребенка", "Более двух детей"]
column_names = ["Процент людей, вернувших кредит вовремя", "Количество людей в категории"]
answer1 = pd.DataFrame(data = table, columns = column_names, index = index)
answer1

,"Процент людей, вернувших кредит вовремя",Количество людей в категории
Нет детей,0.924666,14230
Один - два ребенка,0.907011,6861
Более двух детей,0.918421,380


##### Вывод 1: Данные показывают, что есть зависимость между количеством детей и возвратом кредита в срок. Люди без детей или с большим количеством детей в среднем реже меют задолженности.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

Так же, как и в предыдущем случае, рассчитаем процент людей, вернувших кредит вовремя. Прлученную таблицу отсортируем по надежности заемщика для удобства анализа.

In [45]:
info = data.groupby("family_status_id").agg({"debt": ["mean" ,"count"]})
table = info.to_numpy()
table[:,0] = 1 - table[:,0]
index = data_dict_family_status.to_numpy()[:,1]
column_names = ["Процент людей, вернувших кредит вовремя", "Количество людей в категории"]
answer2 = pd.DataFrame(data = table, columns = column_names, index = index)
answer2 = answer2.sort_values(by = "Процент людей, вернувших кредит вовремя")
answer2

,"Процент людей, вернувших кредит вовремя",Количество людей в категории
Не женат / не замужем,0.902491,2810.0
гражданский брак,0.906798,4163.0
женат / замужем,0.924579,12344.0
в разводе,0.928870,1195.0
вдовец / вдова,0.934307,959.0


##### Вывод 2: Исходя из данной таблицы можно сделать вывод, что семейное положение влияет на возврат кредита в срок. Однако можно сделать вывод, что люди из категории `Не женат / не замужем` и `гражданский брак` в среднем чуть менее надежны.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Аналогично предыдущим шагам рассчитаем надежность заемщика в зависимости от его уровня дохода:

In [46]:
info = data.groupby("total_income_category").agg({"debt": ["mean" ,"count"]})
table = info.to_numpy()
table[:,0] = 1 - table[:,0]
index = info.index.to_numpy()
column_names = ["Процент людей, вернувших кредит вовремя", "Количество людей в категории"]
answer3 = pd.DataFrame(data = table, columns = column_names, index = index)
answer3 = answer3.sort_values(by = "Процент людей, вернувших кредит вовремя")
answer3

,"Процент людей, вернувших кредит вовремя",Количество людей в категории
E,0.909091,22.0
C,0.915175,16033.0
A,0.920000,25.0
B,0.929379,5041.0
D,0.940000,350.0


##### Вывод 3: Самыми надежными заемщиками оказались люди из категории `D`.  Зависимость между уровнем дохода и возвратом кредита в срок есть. Однако исходная таблица содержит очень мало данных для категорий `E` и `A`, поэтому данная таблица не точная и нуждается в дополнительных данных.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

Рассчитаем надежность заемщика в зависимости от цели кредита:

In [47]:
info = data.groupby("purpose_category").agg({"debt": ["mean" ,"count"]})
table = info.to_numpy()
table[:,0] = 1 - table[:,0]
index = info.index.to_numpy()
column_names = ["Процент людей, вернувших кредит вовремя", "Количество людей в категории"]
answer4 = pd.DataFrame(data = table, columns = column_names, index = index)
answer4 = answer4.sort_values(by = "Процент людей, вернувших кредит вовремя")
answer4

,"Процент людей, вернувших кредит вовремя",Количество людей в категории
операции с автомобилем,0.906453,4308.0
получение образования,0.907823,4014.0
проведение свадьбы,0.920343,2335.0
операции с недвижимостью,0.927686,10814.0


##### Вывод 4: Люди, берущие кредит на получение образования или операции с автоиобилем в среднем являются менее надежными заемщиками.

## Общий вывод:

На основании проделанной работы, можно сделать вывод, что любой из рассмотренных факторов (количество детей, семейное положение, уровень дохода, цель кредита) влияет на возврат кретиа вовремя. Однако это влияние порядка 2-4 процентов. Это можно объяснить тем, что в данном наборе данных рассматривались люди с уже одобренным кредитом, поэтому подавляющее большенство из них зведомо были надежными заемщиками. Влияние количества детей можно объяснит так: людям без детей не надо тратить деньги на их образование, еду, одежду или на какие-либо спонтанные расходы, в то же время люди с большим количеством детей уже имею опыт и могу заранее примерно предугадать и рассчитать сумму, которую они потратят на ребенка в ближайшее время. Влияние семейного положения можно объяснить так: люди, состоящие в браке могут оплачивать кредит вместе, что более надежно, чем если бы кредит оплачивал один человек. Вдовы/вдовцы скорее всего получают какую-либо помощь со стороны государства, поэтому являются более надежными заемщиками. Влияние уровня дохода объяснить довольно сложно, люди с наименьшим доходом являются наименее надежными, что вполне закономерно, однако люди со средним, а не с высоким доходом оказались наиболее надежными. Возможно это связано с недостатком данных (людей в некоторых категориях очень мало) или люди с высоким доходом имеют несколько кредитов в разных банках для развития каких-либо идей, поэтому оказываются в среднем менее надежными, чем люди со средним доходом. Влияние целей кредита объясняется тем, что кредит на образование или автомобиль обычно берут молодые люди, не имеющие стабильного большого дохода или работы, в то время как кредит на недвижимость или свадьбу берут уже вставшие на ноги люди.